In [ ]:
# Units
from brian2 import *
print(20*volt)
print(1000*amp)
print(1e6*volt)
print(1000*namp)
print(10*nA*5*Mohm)


In [ ]:
# Simple model
start_scope()

tau = 10*ms
eqs = 'dv/dt = (1-v)/tau : 1' # Solution: v = 1 - exp(-t/tau)

G = NeuronGroup(1, eqs)

print('Before v = %s' % G.v[0])
run(100*ms)
print('After v = %s' % G.v[0])

In [ ]:
# V graph
start_scope()

G = NeuronGroup(1, eqs, method='exact')
M = StateMonitor(G, 'v', record=True)

run(30*ms)

plot(M.t/ms, M.v[0])
xlabel('Time (ms)')
ylabel('v')

In [ ]:
# V graph with expected behaviour
start_scope()

G = NeuronGroup(1, eqs, method='exact')
M = StateMonitor(G, 'v', record=0) # Neuron 0

run(30*ms)

plot(M.t/ms, M.v[0], label='Brian') # Brian simulation
plot(M.t/ms, 1-exp(-M.t/tau), 'C1--',label='Analytic') # Expected behaviour
xlabel('Time (ms)')
ylabel('v')
legend()

In [ ]:
# Euler method
start_scope()

tau = 10*ms
eqs = 'dv/dt = (sin(2*pi*100*Hz*t)-v)/tau : 1'

G = NeuronGroup(1, eqs, method='euler')
M = StateMonitor(G, 'v', record=0) # Neuron 0
G.v = 5 # Initial value

run(60*ms)

plot(M.t/ms, M.v[0])
xlabel('Time (ms)')
ylabel('v')

In [ ]:
# Spikes
start_scope()

tau = 10*ms
eqs = 'dv/dt = (1-v)/tau : 1' # Solution: v = 1 - exp(-t/tau)

G = NeuronGroup(1, eqs, threshold='v>0.8', reset='v=0', method='exact') # Spike fired at v>0.8 and reset
M = StateMonitor(G, 'v', record=0) # Neuron 0

run(50*ms)

plot(M.t/ms, M.v[0])
xlabel('Time (ms)')
ylabel('v')

In [ ]:
# Spike monitor
start_scope()

G = NeuronGroup(1, eqs, threshold='v>0.8', reset='v=0', method='exact') # Spike fired at v>0.8 and reset
spikemon = SpikeMonitor(G)

run(50*ms)

print('Spike times: %s' % spikemon.t[:])

In [ ]:
# Spike graph
start_scope()

G = NeuronGroup(1, eqs, threshold='v>0.8', reset='v=0', method='exact') # Spike fired at v>0.8 and reset
statemon = StateMonitor(G, 'v', record=0)
spikemon = SpikeMonitor(G)

run(50*ms)

plot(statemon.t/ms, statemon.v[0])
for t in spikemon.t:
    axvline(t/ms, ls='--', c='C1', lw=3)
xlabel('Time (ms)')
ylabel('v')

In [ ]:
# Refractoriness
start_scope()

tau = 10*ms
eqs = 'dv/dt = (1-v)/tau : 1 (unless refractory)' # Keep v constant during refractory period

G = NeuronGroup(1, eqs, threshold='v>0.8', reset='v=0', refractory=5*ms, method='exact')
statemon = StateMonitor(G, 'v', record=0)
spikemon = SpikeMonitor(G)

run(50*ms)

plot(statemon.t/ms, statemon.v[0])
for t in spikemon.t:
    axvline(t/ms, ls='--', c='C1', lw=3)
xlabel('Time (ms)')
ylabel('v')

In [ ]:
# Multiple neurons raster plot
start_scope()

N = 100 #
tau = 10*ms
eqs = 'dv/dt = (2-v)/tau : 1'

G = NeuronGroup(N, eqs, threshold='v>1', reset='v=0', method='exact')
G.v = 'rand()' # 0 to 1 random initial value for each neuron
spikemon = SpikeMonitor(G)

run(50*ms)

plot(spikemon.t/ms, spikemon.i, '.k') # Plot spikes of neuron 0 on line 0 etc.
xlabel('Time (ms)')
ylabel('Neuron index')

In [ ]:
# Spike time per neuron index
spikemon.spike_trains()

In [ ]:
# Per neuron params
start_scope()

N = 100 
tau = 10*ms
v0_max = 3.
duration = 1000*ms
eqs = '''
dv/dt = (v0-v)/tau : 1 (unless refractory)
v0 : 1
'''

G = NeuronGroup(N, eqs, threshold='v>1', reset='v=0', refractory=5*ms, method='exact')
G.v0 = 'i*v0_max/(N-1)' # v0 of each neuron from 0 to v0_max in steps of v0_max/(N-1)
M = SpikeMonitor(G)

run(duration)

figure(figsize=(12,4))
subplot(121)
plot(M.t/ms, M.i, '.k')
xlabel('Time (ms)')
ylabel('Neuron index')

subplot(122)
plot(G.v0, M.count/duration) # Count the number of spikes (per second)
xlabel('v0')
ylabel('Firing rate (sp/s)')


In [ ]:
# Stochastic neurons
start_scope()

N = 100
tau = 10*ms
v0_max = 3.
duration = 1000*ms
sigma = 0.2

eqs = '''
dv/dt = (v0-v)/tau+sigma*xi*tau**-0.5 : 1 (unless refractory)
v0 : 1
'''

G = NeuronGroup(N, eqs, threshold='v>1', reset='v=0', refractory=5*ms, method='euler')
M = SpikeMonitor(G)

G.v0 = 'i*v0_max/(N-1)'

run(duration)

figure(figsize=(12,4))
subplot(121)
plot(M.t/ms, M.i, '.k')
xlabel('Time (ms)')
ylabel('Neuron index')
subplot(122)
plot(G.v0, M.count/duration)
xlabel('v0')
ylabel('Firing rate (sp/s)')

In [ ]:
# Negative refactory value
start_scope()

N = 1000
tau = 10*ms
vr = -70*mV
vt0 = -50*mV
delta_vt0 = 5*mV
tau_t = 100*ms
sigma = 0.5*(vt0-vr)
v_drive = 2*(vt0-vr)
duration = 100*ms

eqs = '''
dv/dt = (v_drive+vr-v)/tau + sigma*xi*tau**-0.5 : volt
dvt/dt = (vt0-vt)/tau_t : volt
'''

reset = '''
v = vr
vt += delta_vt0
'''

G = NeuronGroup(N, eqs, threshold='v>vt', reset=reset, refractory=5*ms, method='euler')
spikemon = SpikeMonitor(G)

G.v = 'rand()*(vt0-vr)+vr'
G.vt = vt0

run(duration)

_ = hist(spikemon.t/ms, 100, histtype='stepfilled', facecolor='k', weights=ones(len(spikemon))/(N*defaultclock.dt))
xlabel('Time (ms)')
ylabel('Instantaneous firing rate (sp/s)')